# Clean Tour de France stage data from 2008 to 2011
This notebook imports stage data gathered from [bikeraceinfo.com](http://www.bikeraceinfo.com) for the years 2008 to 2011, cleans it, and prepares it to be joined with data from 2012 to 2016.

In [1]:
import pandas as pd
import csv
import pickle

### Load in raw data, clean, and pickle ###

In [2]:
tdf_0811 = pd.read_csv("tdf2011_prior.csv",dtype=str,quoting=csv.QUOTE_NONE)

In [3]:
# remove whitespace from column names
new_cols = {col: col.strip() for col in tdf_0811.columns}
tdf_0811.rename(columns=new_cols, inplace=True)

In [4]:
# manually add missing winning time data point
tdf_0811.set_value(35,'winning_time',"4h 52' 42");

In [5]:
# reformat column datatypes
tdf_0811['distance'] = tdf_0811.distance.apply(lambda x: float(x))
tdf_0811['num_climbs'] = tdf_0811.num_climbs.apply(lambda x: int(x))

for col in tdf_0811.columns[9:41]:
    tdf_0811[col] = tdf_0811[col].apply(lambda x: float(x))
    
for col in tdf_0811.columns[42:]:
    tdf_0811[col] = tdf_0811[col].apply(lambda x: float(x))

In [6]:
# convert winning times from strings to floats in minutes
for i in tdf_0811.index:
    try:
        time = tdf_0811.winning_time[i].strip('"').split()
        if len(time) > 2:
            hours = time[0].strip('h')
            mins = time[1].strip("'")
            secs = time[2].strip()
        else:
            hours = 0
            mins = time[0].strip("'")
            secs = time[1].strip()

        total = float(hours) * 60.0 + float(mins) + float(secs)/60.0

        tdf_0811.set_value(i,'total_mins',total)
    except:
        tdf_0811.set_value(i,'total_mins',None)
        print('Row ' + str(i) + ' failed')

In [7]:
# drop old winning time column and rename total minutes to winning time
tdf_0811.drop('winning_time',axis=1,inplace=True)

new_col = {'total_mins': 'winning_time'}
tdf_0811.rename(columns=new_col, inplace=True)

In [8]:
# climb categories were incorrectly encoded, correcting this
cols = ['climb_0_category',
       'climb_1_category',
       'climb_2_category',
       'climb_3_category',
        'climb_4_category',
       'climb_5_category',
       'climb_6_category',
       'climb_7_category']

for i in tdf_0811.index:
    num_climbs = tdf_0811.num_climbs[i]
    
    if num_climbs == 0:
        pass
    else:
        for col in cols[:int(num_climbs)+1]:
            cat = tdf_0811[col][i]
            if cat == 0:
                cat = 5
            elif cat == 1:
                cat = 4
            elif cat == 2:
                cat = 3
            elif cat == 3:
                cat = 2
            elif cat == 4:
                cat = 1
            else:
                cat = 5
                
            tdf_0811.set_value(i,col,cat)
            
        for col in cols[int(num_climbs):]:
            cat = 0
            tdf_0811.set_value(i,col,cat)

In [9]:
# average each stage's climb data and store as new columns
def climb_averages(df,features=['category','length','start','gradient']):
    climb_labels = ['climb_0_',
                    'climb_1_',
                    'climb_2_',
                    'climb_3_',
                    'climb_4_',
                    'climb_5_',
                    'climb_6_',
                    'climb_7_']
    
    for feature in features:
        
        for i in df.index:
            values = [df.loc[i,label+feature] for label in climb_labels]
            num_climbs = df.loc[i,'num_climbs']
            
            if num_climbs > 0:
                avg = sum(values)/float(num_climbs)
            else:
                avg = 0.0
            
            df.set_value(i,feature+'_avg',avg)
     
    return df

tdf_0811 = climb_averages(tdf_0811)

In [10]:
# average starting point for each stage's sprints and store as new column
cols = ['sprint_0_start',
       'sprint_1_start',
       'sprint_2_start']

for i in tdf_0811.index:
    starts = [tdf_0811.loc[i,col] for col in cols]
    num_starts= [start for start in starts if start != 0]
    if len(num_starts) > 0:
        avg_start = sum(starts)/len(num_starts)
    else:
        avg_start = 0.0
    tdf_0811.set_value(i, 'num_sprints',len(num_starts))
    tdf_0811.set_value(i,'sprint_start_avg',avg_start)

In [11]:
# reformat week day and month data to match 2012-2016 data
tdf_0811['week_day'] = tdf_0811.week_day.apply(lambda x: x.upper())
tdf_0811['month'] = tdf_0811.month.apply(lambda x: x.upper())

In [12]:
# pickle cleaned data
pickle.dump(tdf_0811,open('tdf_0811_full.p','wb'))

In [13]:
# drop extraneous columns
tdf_0811.drop(['climb_0_start', 'climb_0_category',
       'climb_0_gradient', 'climb_0_length', 'climb_1_start',
       'climb_1_category', 'climb_1_gradient', 'climb_1_length',
       'climb_2_start', 'climb_2_category', 'climb_2_gradient',
       'climb_2_length', 'climb_3_start', 'climb_3_category',
       'climb_3_gradient', 'climb_3_length', 'climb_4_start',
       'climb_4_category', 'climb_4_gradient', 'climb_4_length',
       'climb_5_start', 'climb_5_category', 'climb_5_gradient',
       'climb_5_length', 'climb_6_start', 'climb_6_category',
       'climb_6_gradient', 'climb_6_length', 'climb_7_start',
       'climb_7_category', 'climb_7_gradient', 'climb_7_length',
       'sprint_0_start', 'sprint_1_start', 'sprint_2_start'],axis=1,inplace=True)

In [14]:
tdf_0811.head()

,year,stage,week_day,month,day,start,finish,distance,num_climbs,winning_time,category_avg,length_avg,start_avg,gradient_avg,num_sprints,sprint_start_avg
0,2011,1,SATURDAY,JULY,2,Passage du Gois La Barre de Monts,Mont des Alouettes Les Herbiers,191.5,1,244.516667,1.0,2.2,191.50,4.7,1.0,87.0
1,2011,2,SUNDAY,JULY,3,Les Essarts,Les Essarts,23.0,0,24.800000,0.0,0.0,0.00,0.0,0.0,0.0
2,2011,3,MONDAY,JULY,4,Olonne sur Mer,Redon,198.0,1,280.350000,1.0,1.1,143.00,4.9,1.0,104.0
3,2011,4,TUESDAY,JULY,5,Lorient,MÃ»r de Bretagne,172.5,2,251.650000,1.5,1.8,125.75,6.4,1.0,92.5
4,2011,5,WEDNESDAY,JULY,6,Carhaix,Cap FrÃ©hel,164.5,1,218.533333,1.0,2.3,45.50,5.1,1.0,87.0


In [15]:
# pickle for later
pickle.dump(tdf_0811,open('tdf_0811_geo.p','wb'))